In [ ]:
import sys
!{sys.executable} -m pip install duckdb-engine
!{sys.executable} -m pip install ipython-sql

In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import duckdb, sqlalchemy

In [ ]:
%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Research Question: Does there exist correlation between the popularity of Pitbull’s music (measured through awards and spots on Billboard Hot 100) with American unemployment since his breakthrough in 2009 to the present day?

Source datasets:
https://www.kaggle.com/datasets/dhruvildave/billboard-the-hot-100-songs
https://en.wikipedia.org/wiki/List_of_awards_and_nominations_received_by_Pitbull
https://www.bls.gov/cps/tables.htm

Billboard Hot 100 Data Cleaning:

In [ ]:
billboard_df = pd.read_csv('charts.csv')
print (billboard_df.head())

         date  rank           song                         artist  last-week  \
0  2021-11-06     1     Easy On Me                          Adele        1.0   
1  2021-11-06     2           Stay  The Kid LAROI & Justin Bieber        2.0   
2  2021-11-06     3  Industry Baby        Lil Nas X & Jack Harlow        3.0   
3  2021-11-06     4     Fancy Like                   Walker Hayes        4.0   
4  2021-11-06     5     Bad Habits                     Ed Sheeran        5.0   

   peak-rank  weeks-on-board  
0          1               3  
1          1              16  
2          1              14  
3          3              19  
4          2              18  


In [ ]:
pitbull_df= billboard_df.loc[billboard_df['artist'] == 'Pitbull']

In [29]:
pitbull_df.loc[pitbull_df['date'].between('2009-01-01', '2022-10-15')]